In [30]:
import pandas as pd
import sqlite3

pd.set_option('mode.chained_assignment', None)

Создадим соединение с базой данных с помощью sqlite3

In [31]:
connection = sqlite3.connect("../data/checking-logs.sqlite")

In [32]:
datamart = pd.read_sql("""
SELECT 
    c.uid, 
    c.labname,
    c.timestamp AS first_commit_ts,
    MIN(p.datetime) AS first_view_ts
FROM checker c
LEFT JOIN pageviews p ON c.uid = p.uid
WHERE c.status = 'ready'
    AND c.numTrials = 1
    AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND c.uid LIKE 'user_%'
GROUP BY c.uid, c.labname, c.timestamp
""", connection)
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,None
136,user_8,laba05,2020-05-02 13:28:07.705193,None
137,user_8,laba06,2020-05-16 17:56:15.755553,None
138,user_8,laba06s,2020-05-16 20:01:07.900727,None


first_commit_ts и first_view_ts преобразуем в datetime64[ns]

In [33]:
datamart['first_commit_ts'] = pd.to_datetime(datamart['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(datamart['first_view_ts'])

Создадим два датафрейма: test и control

In [34]:
test = pd.DataFrame()
control = pd.DataFrame()

Датафрейм test должен содержать пользователей со значениями в "first_view_ts"

In [35]:
test = datamart[datamart['first_view_ts'].notna()]
test.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136


Датафрейм control должен содержать пользователей с отсутствующими значениями в "first_view_ts".
Заменим отсутствующие значения в таблице control на среднее значение first_view_ts пользователей из test. 

In [36]:
control = datamart[datamart['first_view_ts'].isna()]
control['first_view_ts'] = test['first_view_ts'].mean()
control.head()

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783552
13,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783552
14,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
15,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783552
16,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783552


Сохраним обе таблицы в базе данных

In [37]:
test.to_sql('test', connection, index=False, if_exists='replace')
control.to_sql('control', connection, index=False, if_exists='replace')

connection.commit()
connection.close()